In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
df_train_original = pd.read_csv('../data/drugs_train.csv')
df_train = df_train_original.copy()

df_train.describe(include='all')

,Age,Sex,BP,Cholesterol,Na,K,Drug
count,160.000000,160,160,160,160.000000,160.000000,160
unique,NaN,2,3,2,NaN,NaN,5
top,NaN,M,HIGH,HIGH,NaN,NaN,drugY
freq,NaN,81,60,82,NaN,NaN,73
mean,44.731250,NaN,NaN,NaN,0.700157,0.050815,NaN
std,16.975647,NaN,NaN,NaN,0.116932,0.017298,NaN
min,15.000000,NaN,NaN,NaN,0.500169,0.020042,NaN
25%,31.000000,NaN,NaN,NaN,0.592570,0.035392,NaN
50%,45.000000,NaN,NaN,NaN,0.724923,0.050363,NaN
75%,59.250000,NaN,NaN,NaN,0.794657,0.066203,NaN


In [ ]:
df_train['Sex'] = df_train["Sex"].map({"F": 0, "M": 1})
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
bp_ohe = ohe.fit_transform(df_train[['BP']])  # columnas 0/1

# Nombres de columnas y merge al DataFrame
bp_cols = ohe.get_feature_names_out(['BP'])
df_train[bp_cols] = bp_ohe
df_train



AttributeError: 'csr_matrix' object has no attribute 'get_feature_names_out'